In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('SCALE불량.csv', encoding='cp949')
df.head()

,plate_no,rolling_date,scale,spec_long,spec_country,steel_kind,pt_thick,pt_width,pt_length,hsb,...,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_method,rolling_temp,descaling_count,work_group
0,PLT_1001,03JAN2023:07:07:53,양품,AB/EH32-TM,미국,T,32,3700,15100,적용,...,1열,1144,116,1133,59,259,TMCP(온도제어),934,8,1조
1,PLT_1002,03JAN2023:07:21:22,양품,AB/EH32-TM,미국,T,32,3700,15100,적용,...,2열,1144,122,1135,53,238,TMCP(온도제어),937,8,1조
2,PLT_1003,03JAN2023:07:31:15,양품,NV-E36-TM,영국,T,33,3600,19200,적용,...,1열,1129,116,1121,55,258,TMCP(온도제어),889,8,1조
3,PLT_1004,03JAN2023:07:41:01,양품,NV-E36-TM,영국,T,33,3600,19200,적용,...,2열,1152,125,1127,68,266,TMCP(온도제어),885,8,1조
4,PLT_1005,03JAN2023:07:52:40,양품,BV-EH36-TM,프랑스,T,38,3100,13300,적용,...,1열,1140,134,1128,48,246,TMCP(온도제어),873,8,1조


In [6]:
# rolling_data를 datetime 형식으로 변환 
df['rolling_date'] = pd.to_datetime(df['rolling_date'], format='%d%b%Y:%H:%M:%S')
df.head(10)

,plate_no,rolling_date,scale,spec_long,spec_country,steel_kind,pt_thick,pt_width,pt_length,hsb,...,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_method,rolling_temp,descaling_count,work_group
0,PLT_1001,2023-01-03 07:07:53,양품,AB/EH32-TM,미국,T,32,3700,15100,적용,...,1열,1144,116,1133,59,259,TMCP(온도제어),934,8,1조
1,PLT_1002,2023-01-03 07:21:22,양품,AB/EH32-TM,미국,T,32,3700,15100,적용,...,2열,1144,122,1135,53,238,TMCP(온도제어),937,8,1조
2,PLT_1003,2023-01-03 07:31:15,양품,NV-E36-TM,영국,T,33,3600,19200,적용,...,1열,1129,116,1121,55,258,TMCP(온도제어),889,8,1조
3,PLT_1004,2023-01-03 07:41:01,양품,NV-E36-TM,영국,T,33,3600,19200,적용,...,2열,1152,125,1127,68,266,TMCP(온도제어),885,8,1조
4,PLT_1005,2023-01-03 07:52:40,양품,BV-EH36-TM,프랑스,T,38,3100,13300,적용,...,1열,1140,134,1128,48,246,TMCP(온도제어),873,8,1조
5,PLT_1006,2023-01-03 08:01:55,양품,BV-EH36-TM,프랑스,T,38,3100,13300,적용,...,2열,1143,127,1128,57,255,TMCP(온도제어),874,8,1조
6,PLT_1007,2023-01-03 08:14:02,양품,BV-EH36-TM,프랑스,T,38,3100,16700,적용,...,1열,1138,126,1130,50,278,TMCP(온도제어),878,8,1조
7,PLT_1008,2023-01-03 08:24:12,양품,BV-EH36-TM,프랑스,T,38,3100,16700,적용,...,2열,1139,126,1131,52,272,TMCP(온도제어),870,8,1조
8,PLT_1009,2023-01-03 08:38:23,양품,BV-EH36-TM,프랑스,T,38,3100,16700,적용,...,1열,1127,126,1122,52,241,TMCP(온도제어),873,8,1조
9,PLT_1010,2023-01-03 08:45:58,불량,AB/EH36-TM,미국,T,38,3100,16200,적용,...,2열,1135,119,1124,73,290,TMCP(온도제어),881,7,1조


In [7]:
df.columns

Index(['plate_no', 'rolling_date', 'scale', 'spec_long', 'spec_country',
       'steel_kind', 'pt_thick', 'pt_width', 'pt_length', 'hsb', 'fur_no',
       'fur_input_row', 'fur_heat_temp', 'fur_heat_time', 'fur_soak_temp',
       'fur_soak_time', 'fur_total_time', 'rolling_method', 'rolling_temp',
       'descaling_count', 'work_group'],
      dtype='object')

In [8]:
df['rolling_method'].value_counts()

CR(제어압연)      840
TMCP(온도제어)    160
Name: rolling_method, dtype: int64

In [ ]:
# 양품/불량 라벨링 (양품:0, 불량:1)
df['SCALE'] = df['SCALE'].str.replace('양품', '0')
df['SCALE'] = df['SCALE'].str.replace('불량', '1')
df['SCALE'] = df['SCALE'].astype(int)

# hsb 라벨링 (적용:0, 미적용1)
df['HSB'] = df['HSB'].str.replace('적용', '0')
df['HSB'] = df['HSB'].str.replace('미적용', '1')
df['HSB'] = df['HSB'].astype(int)

# fur_no 단위 제거 
df['fur_no'] = df['fur_no'].str.replace('호기', '')
df['fur_no'] = df['fur_no'].astype(int)
# fur_input_row 단위 제거 
df['fur_input_row'] = df['fur_input_row'].str.replace('열', '')
df['fur_input_row'] = df['fur_input_row'].astype(int)
# work_group 단위 제거 
df['work_group'] = df['work_group'].str.replace('조', '')
df['work_group'] = df['work_group'].astype(int)

# rolling_method 괄호, 괄호 안 단어 제거 
df['rolling_method'] = df['rolling_method'].str.replace('연속압연', '0')
df['rolling_method'] = df['rolling_method'].str.replace('단조압연', '1')
df['rolling_method'] = df['rolling_method'].astype(int)
